# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Separable Convolution.
   - depthwise's filter shape 爲 (3,3), padding = same
   - pointwise's filters size 爲 128
   - 不需要給 alpha, depth multiplier 參數

##### 載入套件

In [1]:
import warnings
warnings.filterwarnings('ignore')

from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU

Using TensorFlow backend.


#### SeparableConv2D = 先DepthwiseConv2D(use_bias=False) + Conv2D(depth, 1)

In [2]:
from keras.layers import SeparableConv2D

input = Input((64, 64, 3))
output = BatchNormalization()(input)
output = SeparableConv2D(128, (3,3), padding='same')(output)
output = BatchNormalization()(output)

model = Model(inputs=input, outputs=output)
model.summary()







Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 64, 64, 128)       539       
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 128)       512       
Total params: 1,063
Trainable params: 801
Non-trainable params: 262
_________________________________________________________________


##### 定義 Separable Convolution 函數 (請在此實做)

In [3]:
def SeparableConv(input):
    '''
    Args:
        input: input tensor
    Output:
        output: output tensor
    '''
    x = DepthwiseConv2D((3,3), padding='same', depth_multiplier=1, use_bias=False)(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(128, (1,1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


##### 建構模型

In [4]:
input = Input((64, 64, 3))
output = SeparableConv(input)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 64, 64, 3)         27        
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       512       
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 128)       512       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64, 64, 128)       0   

#### DepthwiseConv2D 各層進行個別的Conv
#### kernel(3x3)x pre_depth(3) = 27

In [5]:
input = Input((64, 64, 3))
output = DepthwiseConv2D((3,3), padding='same', depth_multiplier=1, use_bias=False)(input)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (None, 64, 64, 3)         27        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


#### DepthwiseConv2D 各層進行個別的Conv
#### kernel(3x3)x pre_depth(3) + pre_depth_bias(3) = 30

In [6]:
input = Input((64, 64, 3))
output = DepthwiseConv2D((3,3), padding='same', depth_multiplier=1, use_bias=True)(input)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
depthwise_conv2d_3 (Depthwis (None, 64, 64, 3)         30        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


#### DepthwiseConv2D 各層進行個別的Conv
#### ( kernel(3x3)x pre_depth(3) + pre_depth_bias(3) ) * 每層2個kerenl = 60

In [7]:
input = Input((64, 64, 3))
output = DepthwiseConv2D((3,3), padding='same', depth_multiplier=2, use_bias=True)(input)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
depthwise_conv2d_4 (Depthwis (None, 64, 64, 6)         60        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________
